# 4.2 The RSA Algorithm

## Exercises 4.2

In [1]:
%%capture
%run ./4_1_more_number_theory.ipynb
from sympy import divisors, nextprime
from sympy.core.numbers import mod_inverse
from __main__ import phi as euler_phi



### 1. Demonstrate the importance of using extremely large numbers when generating keys by deducing the private keys of the following individuals from their relatively small public keys.

|   Person   |    $n$    |  $e$  |
|:----------:|:---------:|:-----:|
|     A      | 98662273  | 1313  |
|     B      | 99633329  | 2791  |
|     C      | 222561187 | 52107 |

We start by factoring $n$, then computing $\varphi(n)$ and then we find $d$, the multiplicative inverse of $e \pmod{\varphi(n)}$

In [17]:
def crack_trivial_rsa(n, e):
    """
    Cracks a trivial RSA encryption scheme by calculating
    Euler's totient function (`phi`) and derives the modular
    inverse of `e` modulo `phi`.
    """
    _phi = euler_phi(n)
    return mod_inverse(e, _phi)

print(f"Private key d, for Person A: {crack_trivial_rsa(98662273, 1313)}")
print(f"Private key d, for Person B: {crack_trivial_rsa(99633329, 2791)}")
print(f"Private key d, for Person C: {crack_trivial_rsa(222561187, 52107)}")

Private key d, for Person A: 90148577
Private key d, for Person B: 92149591
Private key d, for Person C: 213737507


Note, finding $d$ took less than 10ms for all three examples combined when run on my laptop.


### 2. Generate a set of public and private keys for a group of 5 individuals. Use the same values of $p$ and $q$ for each person in the group.

In [3]:
# Start by picking p and q
p = nextprime(1234567)
q = nextprime(7654321)
print(f"p = {p};  q = {q}")
# then n is the product of p and q, and phi is (p-1)(q-1)
n = p*q
phi = (p-1)*(q-1)
print(f"n = {n};  phi = {phi}")
# now find the divisors of phi so we can pick numbers in range 1..phi that are coprime with phi
# these will be the choices for e
phi_divisors = divisors(phi)
# there's probably alot of them.  Let's just pick some from the middle
i = len(phi_divisors)//2
slice_of_phi_divisors = phi_divisors[i:i+5]
print(f"selected divisors of phi = {slice_of_phi_divisors}")
# we can probably assume that these are not consecutive, so adding 1 to each
# should give us an invertible encryption key (number e with gcd(e,phi) = 1)
# if we're wrong we wont be able to invert and we can just pick another number
e_list = [div+1 for div in slice_of_phi_divisors]
print(f"list of e's = {e_list}")
# now we just find the multiplicative inverse of each
d_list = [mod_inverse(e, phi) for e in e_list]
print(f"list of d's = {d_list}")
print("----------------------------------------")
# finally, output the public, private key pairs
for i in range(len(e_list)):
    print(f"\tPublic key #{i+1}:  (e={e_list[i]}, n={n})")
    print(f"\tPrivate key: (d={d_list[i]})")
    print("\t----------------------------------------")

p = 1234577;  q = 7654337
n = 9449868410449;  phi = 9449859521536
selected divisors of phi = [3254048, 3348772, 3365488, 3718912, 3827168]
list of e's = [3254049, 3348773, 3365489, 3718913, 3827169]
list of d's = [5810841372897, 6232744492717, 4905693486737, 8893298308353, 825439767073]
----------------------------------------
	Public key #1:  (e=3254049, n=9449868410449)
	Private key: (d=5810841372897)
	----------------------------------------
	Public key #2:  (e=3348773, n=9449868410449)
	Private key: (d=6232744492717)
	----------------------------------------
	Public key #3:  (e=3365489, n=9449868410449)
	Private key: (d=4905693486737)
	----------------------------------------
	Public key #4:  (e=3718913, n=9449868410449)
	Private key: (d=8893298308353)
	----------------------------------------
	Public key #5:  (e=3827169, n=9449868410449)
	Private key: (d=825439767073)
	----------------------------------------



### 3. Start with $p = 19$ and $q = 41$.

#### (a) Compute a set of private and public keys.

In [4]:
p = 19
q = 41
n = p*q
phi = (p-1)*(q-1)
divs = divisors(phi)
print(divs)

[1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 15, 16, 18, 20, 24, 30, 36, 40, 45, 48, 60, 72, 80, 90, 120, 144, 180, 240, 360, 720]


In [5]:
e = 31  # just picking some number less than phi that is not a divisor
d = mod_inverse(e, phi)
print(f"e = {e}; d = {d}; n = {n} ")

e = 31; d = 511; n = 779 


#### (b) Encipher the message 436.

In [6]:
m = 436
c = pow(m, e, n)
print(c)

398


#### (c) Decipher your answer to part (b) of this problem.

In [7]:
m_prime = pow(c, d, n)
print(m_prime)

436



#### 4. Solve the following equations for $x$;

In [8]:
# for numbers this small, we can just iterate 1..n to find the answers
def discrete_log_mod(base:int, modulus:int, y:int):
    """
    Calculate the discrete logarithm modulo a given modulus.
    """
    for x in range(1, modulus):
        if pow(base, x, modulus) == y:
            return x
    return None

#### (a) $11^x \pmod{29} = 16$

In [9]:
discrete_log_mod(11, 29, 16)

8

#### (b) $13^x \pmod{19} = 2$

In [10]:
discrete_log_mod(13, 19, 2)

11

#### (c) $5^x \pmod{37} = 29$

In [11]:
discrete_log_mod(5, 37, 29)

15

### 5. For $m$ any real number, one usually thinks of $m^8$ as $m \cdot m \cdot m \cdot m \cdot m \cdot m \cdot m \cdot m$, an expression involving 7 multiplications. A more efficient way to compute $m^8$ however, would be to first compute $m^2 = m \cdot m$, and then perform the three multiplications: $m^2 \cdot m^2 \cdot m^2 \cdot  m^2$. This observation lies at the heart of the so-called square and multiply algorithm which can be used in connection with evaluating $m^e \pmod{n}$. Using this technique, how many multiplications would be required to compute:

An algorithmic description of "square and multiply" is

1. Convert the exponent to binary. Ignore leading zero's.  (left most digit will be 1)
2. Initialize the result with the base
3. Process each binary digit (from left to right) after the left most 1:
4. For each binary digit:
    * Square the current result.
    * If the digit is 1, multiply the result by the base


Using this definition, the number of multiplications can be derived as

$$
  (k − 1) + (h − 1)
$$

or, equivilantly

$$
  k + h − 2
$$

where

* $k$ = number of bits in the binary representation of the exponent
* $h$ = number of 1’s in the binary representation of the exponent

In [12]:
def bits_required(n):
    return n.bit_length()

def count_ones(n):
    return bin(n).count('1')

def mults_required_for_exponent(n: int) -> int:
    k = bits_required(n)
    h = count_ones(n)
    return k + h - 2

### (a) $m^{16}$

In [13]:
mults_required_for_exponent(16)

4

### (b) $m^{35}$

In [14]:
mults_required_for_exponent(35)

7

### (c) $m^{103}$

In [15]:
mults_required_for_exponent(103)

10

Note that the exercise question as asked does not show the the proper (i.e. optimal) application of the square and multiply algorithm for an exponent of $8$.

$8$ in binary is $1000$

1. Start with $m$, then moving left to right
2. Next bit is $0$, so square $m$ to get $m^2$ (1st multiplication)
3. Next bit is $0$, so square $m^2$ to get $m^4$ (2nd multiplication)
4. Next (and last) bit is $0$, so square $m^4$ to get $m^8$ (3nd multiplication)

So, only $3$ multiplications are required to compute $m^8$, which matches what `mults_required_for_exponent` computes.

In [16]:
mults_required_for_exponent(8)

3